### !! Open this notebook on colab for faster execution !!

In [1]:
!rm -rf mercury_surfer
!git clone https://github.com/thisisanshrastogi/mercury_surfer.git


Cloning into 'mercury_surfer'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (47/47), done.
^Cceiving objects:  24% (13/54), 2.93 MiB | 382.00 KiB/s  


In [ ]:
!pip install bertopic[all]



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from bertopic import BERTopic
import pandas as pd

df = pd.read_csv('./mercury_surfer/sentiment_classified.csv')


In [ ]:
df = df[df['Stars'] <4].reset_index(drop=True)

In [ ]:
df.tail()

,Review,Date,Stars,Cleaned Reviews,neg,neu,pos,compound,vader_label,star_sentiments
29457,Down almost all the time,2024-01-31,3,almost time,0.000,1.000,0.000,0.0000,neutral,1
29458,The system brings me to a page of the pay. But...,2024-01-29,3,system bring page pay get frozen move forward ...,0.048,0.952,0.000,-0.0516,negative,1
29459,Not easy logging in... very difficult,2024-01-24,3,easy log difficult,0.565,0.435,0.000,-0.6368,negative,1
29460,Bugs while logging in. I need to verify my dev...,2024-01-19,3,bug log need verify device every time save cre...,0.120,0.741,0.139,-0.0547,negative,1
29461,still never fixed ability to send cash from ci...,2024-01-04,3,still never fix ability send cash citizen Bank...,0.131,0.869,0.000,-0.2411,negative,1


In [ ]:
docs = df['Cleaned Reviews'].dropna().tolist()
topic_model = BERTopic(language="english",min_topic_size=20)
topics, probs = topic_model.fit_transform(docs)


In [ ]:
from sentence_transformers import SentenceTransformer
docs = df['Cleaned Reviews'].dropna().tolist()
embedding_model = SentenceTransformer("all-mpnet-base-v2")  # more expressive
topic_model = BERTopic(embedding_model=embedding_model)

In [ ]:
topics, probs = topic_model.fit_transform(docs)


In [ ]:
topics = topic_model.get_topic_info()
# topics is a DataFrame with columns: Topic, Count, Name

# To get top keywords per topic in a cleaner way:
def format_topic_words(topic_id):
    words = topic_model.get_topic(topic_id)
    # words is a list of (word, score) tuples
    return ", ".join([word for word, _ in words])

topics['Top Keywords'] = topics['Topic'].apply(lambda x: format_topic_words(x) if x != -1 else "Outliers")

# Save to CSV
topics.to_csv("bertopic_topics_expressive.csv", index=False)

print(topics[['Topic', 'Count', 'Top Keywords']])

      Topic  Count                                       Top Keywords
0        -1   4712                                           Outliers
1         0    186  good, fair, fubctional, chime, finish, suppose...
2         1    138  suck, sucks, hate, confusing, hell, care, happ...
3         2    120  grapheneo, work, tin, airways, maybe, say, , , , 
4         3    112  notification, push, alert, notify, disable, pa...
...     ...    ...                                                ...
996     995     10  fingerprint, meh, lately, inconsistent, depend...
997     996     10  goddam, washington, mutual, teach, pit, favor,...
998     997     10  stability, msgse, shut, serious, refresh, conn...
999     998     10  slow, kinda, price, hope, functional, unreliab...
1000    999     10  disregard, pony, periodically, dog, specifical...

[1001 rows x 3 columns]
